In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "projects/smtlink/top" :dir :system)
(include-book "centaur/fty/top" :dir :system)


TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "projects/smtlink/top" ...):
The ttag note just printed to the terminal indicates a modification
to ACL2.  To avoid this warning, supply an explicit :TTAGS argument
when including the book "/home/acl2/books/projects/smtlink/top".


TTAG NOTE (for included book): Adding ttag :QUICKLISP.SHELLPOOL from book /home/acl2/books/quicklisp/shellpool.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "projects/smtlink/top" ...):
The ttag note just printed to the terminal indicates a modification
to ACL2.  To avoid this warning, supply an explicit :TTAGS argument
when including the book "/home/acl2/books/projects/smtlink/top".


TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/projects/smtlink/trusted/run.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "projects/smtlink/top" ...):
The ttag note just printed to the terminal indicates a modification
t

In [3]:
; Enable tshell for running Z3
(value-triple (tshell-ensure))

 NIL


In [4]:
; Install SMTLink computed hint for proper Z3 invocation
(add-default-hints '((SMT::SMT-computed-hint clause)))

 ((SMT::SMT-COMPUTED-HINT CLAUSE)
  (SET::PICK-A-POINT-SUBSET-HINT ID CLAUSE
                                 WORLD STABLE-UNDER-SIMPLIFICATIONP))


In [5]:
;; Agent state as FTY product type
;; Using integerp for SMTLink compatibility (maps to Z3 Int)
;; Note: integerp fields work with SMTLink because it treats them as TYPE
;; declarations. We just need to avoid calling integerp as a function.
(fty::defprod agent-state
  ((iteration      integerp :default 0)
   (max-iterations integerp :default 10)
   (token-budget   integerp :default 10000)
   (cost-budget    integerp :default 1000)   ; millicents
   (time-budget    integerp :default 60000)  ; ms
   (file-access    integerp :default 0)      ; 0=none, 1=read, 2=read-write
   (exec-allowed   booleanp :default nil)
   (satisfaction   integerp :default 0)      ; 0-100 (percentage)
   (done           booleanp :default nil))
  :parents (experiment-02))

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.04 seconds (prove: 0.00, print: 0.00, other: 0.04)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [6]:
;; Tool spec as FTY product type
(fty::defprod tool-spec
  ((required-access integerp :default 0)
   (requires-exec   booleanp :default nil)
   (base-cost       integerp :default 0)
   (time-estimate   integerp :default 0)
   (token-estimate  integerp :default 0))
  :parents (experiment-02))

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [7]:
(defconst *access-none* 0)
(defconst *access-read* 1)
(defconst *access-read-write* 2)


Summary
Form:  ( DEFCONST *ACCESS-NONE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *ACCESS-NONE*

Summary
Form:  ( DEFCONST *ACCESS-READ* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *ACCESS-READ*

Summary
Form:  ( DEFCONST *ACCESS-READ-WRITE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *ACCESS-READ-WRITE*


In [8]:
(defconst *min-llm-tokens* 100)
(defconst *min-iteration-cost* 10)
(defconst *min-iteration-time* 1000)
(defconst *satisfaction-threshold* 90)  ; percentage


Summary
Form:  ( DEFCONST *MIN-LLM-TOKENS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MIN-LLM-TOKENS*

Summary
Form:  ( DEFCONST *MIN-ITERATION-COST* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MIN-ITERATION-COST*

Summary
Form:  ( DEFCONST *MIN-ITERATION-TIME* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MIN-ITERATION-TIME*

Summary
Form:  ( DEFCONST *SATISFACTION-THRESHOLD* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SATISFACTION-THRESHOLD*


In [9]:
;; Check if granted access is sufficient
;; The FTY accessors already return integers, so we compare directly.
;; NO ifix needed - the FTY type guarantees integerp.
(define access-sufficient-p ((required integerp) (granted integerp))
  :returns (ok booleanp :rule-classes :type-prescription)
  (<= required granted))


Since ACCESS-SUFFICIENT-P is non-recursive, its admission is trivial.
We observe that the type of ACCESS-SUFFICIENT-P is described by the
theorem 
(OR (EQUAL (ACCESS-SUFFICIENT-P REQUIRED GRANTED)
           T)
    (EQUAL (ACCESS-SUFFICIENT-P REQUIRED GRANTED)
           NIL)).

Computing the guard conjecture for ACCESS-SUFFICIENT-P....

The guard conjecture for ACCESS-SUFFICIENT-P is trivial to prove. 
ACCESS-SUFFICIENT-P is compliant with Common Lisp.

Summary
Form:  ( DEFUN ACCESS-SUFFICIENT-P ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)


ACL2 !>>(DEFTHM BOOLEANP-OF-ACCESS-SUFFICIENT-P
          (B* ((OK (ACCESS-SUFFICIENT-P REQUIRED GRANTED)))
            (BOOLEANP OK))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'ACCESS-SUFFICIENT-P
                                                        ID WORLD))
          :RULE-CLASSES :TYPE-PRESCRIPTION)

ACL2 Observation in ( DEFTHM BOOLEANP-OF-ACCESS-SUFFICIENT-P ...):
Our heuristics choose (ACCES

In [10]:
;; Full permission check: file access + execute
(define tool-permitted-p ((tool tool-spec-p) (st agent-state-p))
  :returns (ok booleanp :rule-classes :type-prescription)
  (b* ((required-access (tool-spec->required-access tool))
       (requires-exec (tool-spec->requires-exec tool))
       (granted-access (agent-state->file-access st))
       (exec-allowed (agent-state->exec-allowed st)))
    (and (access-sufficient-p required-access granted-access)
         (implies requires-exec exec-allowed))))


Since TOOL-PERMITTED-P is non-recursive, its admission is trivial.
We observe that the type of TOOL-PERMITTED-P is described by the theorem
(OR (EQUAL (TOOL-PERMITTED-P TOOL ST) T)
    (EQUAL (TOOL-PERMITTED-P TOOL ST) NIL)).
We used the :type-prescription rule AGENT-STATE->EXEC-ALLOWED$INLINE.

Computing the guard conjecture for TOOL-PERMITTED-P....

The non-trivial part of the guard conjecture for TOOL-PERMITTED-P is

Goal
(IMPLIES (AND (AGENT-STATE-P ST)
              (TOOL-SPEC-P TOOL))
         (AND (TOOL-SPEC-P TOOL)
              (LET ((REQUIRED-ACCESS (TOOL-SPEC->REQUIRED-ACCESS TOOL)))
                (AND (TOOL-SPEC-P TOOL)
                     (AGENT-STATE-P ST)
                     (LET ((GRANTED-ACCESS (AGENT-STATE->FILE-ACCESS ST)))
                       (AND (AGENT-STATE-P ST)
                            (INTEGERP REQUIRED-ACCESS)
                            (INTEGERP GRANTED-ACCESS))))))).

Q.E.D.

That completes the proof of the guard theorem for TOOL-PERMITTED-P.
TO

In [11]:
;; Check if budget is sufficient for a tool call
(define tool-budget-sufficient-p ((tool tool-spec-p) (st agent-state-p))
  :returns (ok booleanp :rule-classes :type-prescription)
  (b* ((tool-cost (tool-spec->base-cost tool))
       (tool-time (tool-spec->time-estimate tool))
       (tool-tokens (tool-spec->token-estimate tool))
       (budget-cost (agent-state->cost-budget st))
       (budget-time (agent-state->time-budget st))
       (budget-tokens (agent-state->token-budget st)))
    (and (<= tool-cost budget-cost)
         (<= tool-time budget-time)
         (<= tool-tokens budget-tokens))))


Since TOOL-BUDGET-SUFFICIENT-P is non-recursive, its admission is trivial.
We observe that the type of TOOL-BUDGET-SUFFICIENT-P is described by
the theorem 
(OR (EQUAL (TOOL-BUDGET-SUFFICIENT-P TOOL ST)
           T)
    (EQUAL (TOOL-BUDGET-SUFFICIENT-P TOOL ST)
           NIL)).

Computing the guard conjecture for TOOL-BUDGET-SUFFICIENT-P....

The non-trivial part of the guard conjecture for TOOL-BUDGET-SUFFICIENT-P
is

Goal
(IMPLIES
 (AND (AGENT-STATE-P ST)
      (TOOL-SPEC-P TOOL))
 (AND
  (TOOL-SPEC-P TOOL)
  (LET ((TOOL-COST (TOOL-SPEC->BASE-COST TOOL)))
   (AND
    (TOOL-SPEC-P TOOL)
    (LET ((TOOL-TIME (TOOL-SPEC->TIME-ESTIMATE TOOL)))
     (AND
      (TOOL-SPEC-P TOOL)
      (LET ((TOOL-TOKENS (TOOL-SPEC->TOKEN-ESTIMATE TOOL)))
       (AND
          (AGENT-STATE-P ST)
          (LET ((BUDGET-COST (AGENT-STATE->COST-BUDGET ST)))
            (AND (AGENT-STATE-P ST)
                 (LET ((BUDGET-TIME (AGENT-STATE->TIME-BUDGET ST)))
                   (AND (AGENT-STATE-P ST)
   

In [12]:
;; Can we invoke this tool?
(define can-invoke-tool-p ((tool tool-spec-p) (st agent-state-p))
  :returns (ok booleanp :rule-classes :type-prescription)
  (and (tool-permitted-p tool st)
       (tool-budget-sufficient-p tool st)))


Since CAN-INVOKE-TOOL-P is non-recursive, its admission is trivial.
We observe that the type of CAN-INVOKE-TOOL-P is described by the theorem
(OR (EQUAL (CAN-INVOKE-TOOL-P TOOL ST) T)
    (EQUAL (CAN-INVOKE-TOOL-P TOOL ST)
           NIL)).
We used the :type-prescription rule BOOLEANP-OF-TOOL-BUDGET-SUFFICIENT-P.

Computing the guard conjecture for CAN-INVOKE-TOOL-P....

The guard conjecture for CAN-INVOKE-TOOL-P is trivial to prove, given
the :compound-recognizer rules CONSP-WHEN-AGENT-STATE-P and 
CONSP-WHEN-TOOL-SPEC-P.  CAN-INVOKE-TOOL-P is compliant with Common
Lisp.

Summary
Form:  ( DEFUN CAN-INVOKE-TOOL-P ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:COMPOUND-RECOGNIZER CONSP-WHEN-TOOL-SPEC-P)
        (:TYPE-PRESCRIPTION BOOLEANP-OF-TOOL-BUDGET-SUFFICIENT-P))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOOLEANP-OF-CAN-INVOKE-TOOL-P
          (B* ((OK (CAN-INVOKE-TOOL-P TOOL ST)))
            (BOOLEANP OK))
          :H

In [13]:
;; Must respond: out of budget or done
(define must-respond-p ((st agent-state-p))
  :returns (must booleanp :rule-classes :type-prescription)
  (b* ((done (agent-state->done st))
       (iter (agent-state->iteration st))
       (max-iter (agent-state->max-iterations st))
       (tokens (agent-state->token-budget st))
       (cost (agent-state->cost-budget st))
       (time-ms (agent-state->time-budget st)))
    (or done
        (>= iter max-iter)
        (< tokens *min-llm-tokens*)
        (< cost *min-iteration-cost*)
        (< time-ms *min-iteration-time*))))


Since MUST-RESPOND-P is non-recursive, its admission is trivial.  We
observe that the type of MUST-RESPOND-P is described by the theorem
(OR (EQUAL (MUST-RESPOND-P ST) T) (EQUAL (MUST-RESPOND-P ST) NIL)).
We used the :type-prescription rule AGENT-STATE->DONE$INLINE.

Computing the guard conjecture for MUST-RESPOND-P....

The non-trivial part of the guard conjecture for MUST-RESPOND-P is

Goal
(IMPLIES
 (AGENT-STATE-P ST)
 (AND
  (AGENT-STATE-P ST)
  (LET ((DONE (AGENT-STATE->DONE ST)))
   (AND
    (AGENT-STATE-P ST)
    (LET ((ITER (AGENT-STATE->ITERATION ST)))
     (AND (AGENT-STATE-P ST)
          (LET ((MAX-ITER (AGENT-STATE->MAX-ITERATIONS ST)))
            (AND (AGENT-STATE-P ST)
                 (LET ((TOKENS (AGENT-STATE->TOKEN-BUDGET ST)))
                   (AND (AGENT-STATE-P ST)
                        (LET ((COST (AGENT-STATE->COST-BUDGET ST)))
                          (AND (AGENT-STATE-P ST)
                               (LET ((TIME-MS (AGENT-STATE->TIME-BUDGET ST)))
  

In [14]:
;; Should continue: has budget and below satisfaction threshold
(define should-continue-p ((st agent-state-p))
  :returns (cont booleanp :rule-classes :type-prescription)
  (b* ((sat (agent-state->satisfaction st)))
    (and (not (must-respond-p st))
         (< sat *satisfaction-threshold*))))


Since SHOULD-CONTINUE-P is non-recursive, its admission is trivial.
We observe that the type of SHOULD-CONTINUE-P is described by the theorem
(OR (EQUAL (SHOULD-CONTINUE-P ST) T) (EQUAL (SHOULD-CONTINUE-P ST) NIL)).

Computing the guard conjecture for SHOULD-CONTINUE-P....

The non-trivial part of the guard conjecture for SHOULD-CONTINUE-P
is

Goal
(IMPLIES (AGENT-STATE-P ST)
         (AND (AGENT-STATE-P ST)
              (LET ((SAT (AGENT-STATE->SATISFACTION ST)))
                (AND (AGENT-STATE-P ST)
                     (OR (MUST-RESPOND-P ST)
                         (RATIONALP SAT)))))).

Q.E.D.

That completes the proof of the guard theorem for SHOULD-CONTINUE-P.
SHOULD-CONTINUE-P is compliant with Common Lisp.

Summary
Form:  ( DEFUN SHOULD-CONTINUE-P ...)
Rules: ((:DEFINITION NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  28


ACL2 !>>(DEFTHM BOOLEANP-OF-SHOULD-CONTINUE-P
          (B* ((C

In [15]:
;; Deduct tool costs from state
;; Note: Use (max 0 ...) to ensure non-negative, avoiding ifix
(define deduct-tool-cost ((tool tool-spec-p) (st agent-state-p))
  :returns (new-st agent-state-p)
  (b* ((tool-cost (tool-spec->base-cost tool))
       (tool-time (tool-spec->time-estimate tool))
       (tool-tokens (tool-spec->token-estimate tool))
       (budget-cost (agent-state->cost-budget st))
       (budget-time (agent-state->time-budget st))
       (budget-tokens (agent-state->token-budget st))
       ;; Use max 0 to ensure non-negative (max is supported by SMTLink)
       (new-cost (max 0 (- budget-cost tool-cost)))
       (new-time (max 0 (- budget-time tool-time)))
       (new-tokens (max 0 (- budget-tokens tool-tokens))))
    (change-agent-state st
                        :cost-budget new-cost
                        :time-budget new-time
                        :token-budget new-tokens)))


Since DEDUCT-TOOL-COST is non-recursive, its admission is trivial.
We observe that the type of DEDUCT-TOOL-COST is described by the theorem
(AND (CONSP (DEDUCT-TOOL-COST TOOL ST))
     (TRUE-LISTP (DEDUCT-TOOL-COST TOOL ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for DEDUCT-TOOL-COST....

The non-trivial part of the guard conjecture for DEDUCT-TOOL-COST is

Goal
(IMPLIES
 (AND (AGENT-STATE-P ST)
      (TOOL-SPEC-P TOOL))
 (AND
  (TOOL-SPEC-P TOOL)
  (LET ((TOOL-COST (TOOL-SPEC->BASE-COST TOOL)))
   (AND
    (TOOL-SPEC-P TOOL)
    (LET ((TOOL-TIME (TOOL-SPEC->TIME-ESTIMATE TOOL)))
     (AND
      (TOOL-SPEC-P TOOL)
      (LET ((TOOL-TOKENS (TOOL-SPEC->TOKEN-ESTIMATE TOOL)))
       (AND
        (AGENT-STATE-P ST)
        (LET ((BUDGET-COST (AGENT-STATE->COST-BUDGET ST)))
         (AND
          (AGENT-STATE-P ST)
          (LET ((BUDGET-TIME (AGENT-STATE->TIME-BUDGET ST)))
           (AND
            (AGENT-STATE-P ST)
            (LET ((BUDGE

In [16]:
;; Increment iteration
(define increment-iteration ((st agent-state-p))
  :returns (new-st agent-state-p)
  (change-agent-state st
                      :iteration (1+ (agent-state->iteration st))))


Since INCREMENT-ITERATION is non-recursive, its admission is trivial.
We observe that the type of INCREMENT-ITERATION is described by the
theorem 
(AND (CONSP (INCREMENT-ITERATION ST)) (TRUE-LISTP (INCREMENT-ITERATION ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for INCREMENT-ITERATION....

The non-trivial part of the guard conjecture for INCREMENT-ITERATION
is

Goal
(IMPLIES
   (AGENT-STATE-P ST)
   (AND (AGENT-STATE-P ST)
        (ACL2-NUMBERP (AGENT-STATE->ITERATION ST))
        (LET ((CHANGE-AGENT-STATE ST)
              (AGENT-STATE->ITERATION (+ 1 (AGENT-STATE->ITERATION ST))))
          (AND (AGENT-STATE-P CHANGE-AGENT-STATE)
               (INTEGERP AGENT-STATE->ITERATION)
               (BOOLEANP (AGENT-STATE->DONE CHANGE-AGENT-STATE))
               (INTEGERP (AGENT-STATE->TIME-BUDGET CHANGE-AGENT-STATE))
               (INTEGERP (AGENT-STATE->TOKEN-BUDGET CHANGE-AGENT-STATE))
               (BOOLEANP (AGENT-STATE->EXEC-ALLOWED CHAN

In [17]:
;; Update satisfaction score
(define update-satisfaction ((new-sat integerp) (st agent-state-p))
  :returns (new-st agent-state-p)
  ;; Clamp to 0-100 using max/min (SMTLink-compatible)
  (b* ((clamped (max 0 (min 100 new-sat))))
    (change-agent-state st :satisfaction clamped)))


Since UPDATE-SATISFACTION is non-recursive, its admission is trivial.
We observe that the type of UPDATE-SATISFACTION is described by the
theorem 
(AND (CONSP (UPDATE-SATISFACTION NEW-SAT ST))
     (TRUE-LISTP (UPDATE-SATISFACTION NEW-SAT ST))).
We used the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for UPDATE-SATISFACTION....

The non-trivial part of the guard conjecture for UPDATE-SATISFACTION
is

Goal
(IMPLIES
    (AND (AGENT-STATE-P ST)
         (INTEGERP NEW-SAT))
    (AND (RATIONALP NEW-SAT)
         (RATIONALP (MIN 100 NEW-SAT))
         (LET ((CLAMPED (MAX 0 (MIN 100 NEW-SAT))))
           (LET ((CHANGE-AGENT-STATE ST)
                 (AGENT-STATE->SATISFACTION CLAMPED))
             (AND (AGENT-STATE-P CHANGE-AGENT-STATE)
                  (INTEGERP (AGENT-STATE->ITERATION CHANGE-AGENT-STATE))
                  (BOOLEANP (AGENT-STATE->DONE CHANGE-AGENT-STATE))
                  (INTEGERP (AGENT-STATE->TIME-BUDGET CHANGE-AGENT-STATE))
               

In [18]:
;; Mark done
(define mark-done ((st agent-state-p))
  :returns (new-st agent-state-p)
  (change-agent-state st :done t))


Since MARK-DONE is non-recursive, its admission is trivial.  We observe
that the type of MARK-DONE is described by the theorem 
(AND (CONSP (MARK-DONE ST)) (TRUE-LISTP (MARK-DONE ST))).  We used
the :type-prescription rule AGENT-STATE.

Computing the guard conjecture for MARK-DONE....

The non-trivial part of the guard conjecture for MARK-DONE is

Goal
(IMPLIES (AGENT-STATE-P ST)
         (LET ((CHANGE-AGENT-STATE ST)
               (AGENT-STATE->DONE T))
           (AND (AGENT-STATE-P CHANGE-AGENT-STATE)
                (INTEGERP (AGENT-STATE->ITERATION CHANGE-AGENT-STATE))
                (BOOLEANP AGENT-STATE->DONE)
                (INTEGERP (AGENT-STATE->TIME-BUDGET CHANGE-AGENT-STATE))
                (INTEGERP (AGENT-STATE->TOKEN-BUDGET CHANGE-AGENT-STATE))
                (BOOLEANP (AGENT-STATE->EXEC-ALLOWED CHANGE-AGENT-STATE))
                (INTEGERP (AGENT-STATE->MAX-ITERATIONS CHANGE-AGENT-STATE))
                (INTEGERP (AGENT-STATE->FILE-ACCESS CHANGE-AGENT-STATE))
  

In [19]:
;; Theorem 1: Permission safety (can-invoke implies permitted)
;; This is proven by Z3 via SMTLink
(defthm permission-safety-smt
  (implies (and (tool-spec-p tool)
                (agent-state-p st)
                (can-invoke-tool-p tool st))
           (tool-permitted-p tool st))
  :hints (("Goal"
           :smtlink
           (:fty (tool-spec agent-state)))))



Using clause-processor Smtlink
Goal'
Goal''
Goal'''
Goal'4'
Subgoal 2
Subgoal 2'
Subgoal 2.2
Subgoal 2.2'
Subgoal 2.2''
Subgoal 2.2'''
Using default SMT-trusted-cp...
; SMT solver: `python /tmp/py_file/smtlink.DbsNS`: 0.13 sec, 0 bytes
Proved!
Subgoal 2.2'4'
Subgoal 2.2'5'
Subgoal 2.1
Subgoal 2.1'
Subgoal 1
Subgoal 1'

Q.E.D.

The storage of PERMISSION-SAFETY-SMT depends upon the :type-prescription
rule BOOLEANP-OF-TOOL-PERMITTED-P.

Summary
Form:  ( DEFTHM PERMISSION-SAFETY-SMT ...)
Rules: ((:DEFINITION ACCESS-SUFFICIENT-P)
        (:DEFINITION CAN-INVOKE-TOOL-P)
        (:DEFINITION NOT)
        (:DEFINITION TOOL-BUDGET-SUFFICIENT-P)
        (:DEFINITION TOOL-PERMITTED-P)
        (:TYPE-PRESCRIPTION BOOLEANP-OF-TOOL-PERMITTED-P))
Hint-events: ((:CLAUSE-PROCESSOR SMT::ADD-HYPO-CP)
              (:CLAUSE-PROCESSOR SMT::EXPAND-CP-FN)
              (:CLAUSE-PROCESSOR SMT::PROCESS-HINT)
              (:CLAUSE-PROCESSOR SMT::REMOVE-HINT-PLEASE)
              (:CLAUSE-PROCESSOR SMT::SMT-T

In [20]:
;; Theorem 2: Budget bounds - deduction keeps budgets non-negative
;; Using >= 0 constraints as these are within the FTY-understood types
(defthm budget-bounds-smt
  (implies (and (tool-spec-p tool)
                (agent-state-p st)
                (>= (agent-state->cost-budget st) 0)
                (>= (agent-state->time-budget st) 0)
                (>= (agent-state->token-budget st) 0))
           (and (>= (agent-state->cost-budget (deduct-tool-cost tool st)) 0)
                (>= (agent-state->time-budget (deduct-tool-cost tool st)) 0)
                (>= (agent-state->token-budget (deduct-tool-cost tool st)) 0)))
  :hints (("Goal"
           :smtlink
           (:fty (tool-spec agent-state)))))


ACL2 Warning [Double-rewrite] in ( DEFTHM BUDGET-BOUNDS-SMT ...): 
In a :REWRITE rule generated from BUDGET-BOUNDS-SMT, equivalence relation
AGENT-STATE-EQUIV$INLINE is maintained at one problematic occurrence
of variable ST in the third hypothesis, but not at any binding occurrence
of ST.  Consider replacing that occurrence of ST in the third hypothesis
with (DOUBLE-REWRITE ST).  See :doc double-rewrite for more information
on this issue.


ACL2 Warning [Double-rewrite] in ( DEFTHM BUDGET-BOUNDS-SMT ...): 
In a :REWRITE rule generated from BUDGET-BOUNDS-SMT, equivalence relation
AGENT-STATE-EQUIV$INLINE is maintained at one problematic occurrence
of variable ST in the fourth hypothesis, but not at any binding occurrence
of ST.  Consider replacing that occurrence of ST in the fourth hypothesis
with (DOUBLE-REWRITE ST).  See :doc double-rewrite for more information
on this issue.


ACL2 Warning [Double-rewrite] in ( DEFTHM BUDGET-BOUNDS-SMT ...): 
In a :REWRITE rule generated from BUDG

In [21]:
;; Theorem 3: Iteration bound guarantees termination
(defthm termination-bound-smt
  (implies (and (agent-state-p st)
                (>= (agent-state->iteration st) (agent-state->max-iterations st)))
           (must-respond-p st))
  :hints (("Goal"
           :smtlink
           (:fty (agent-state)))))


ACL2 Warning [Double-rewrite] in ( DEFTHM TERMINATION-BOUND-SMT ...):
In a :REWRITE rule generated from TERMINATION-BOUND-SMT, equivalence
relation AGENT-STATE-EQUIV$INLINE is maintained at two problematic
occurrences of variable ST in the second hypothesis, but not at any
binding occurrence of ST.  Consider replacing those two occurrences
of ST in the second hypothesis with (DOUBLE-REWRITE ST).  See :doc
double-rewrite for more information on this issue.



Using clause-processor Smtlink
Goal'
Goal''
Goal'''
Goal'4'
Subgoal 2
Subgoal 2'
Subgoal 2.2
Subgoal 2.2'
Subgoal 2.2''
Subgoal 2.2'''
Using default SMT-trusted-cp...
; SMT solver: `python /tmp/py_file/smtlink.P3GJ9`: 0.07 sec, 0 bytes
Proved!
Subgoal 2.2'4'
Subgoal 2.2'5'
Subgoal 2.1
Subgoal 2.1'
Subgoal 1
Subgoal 1'
Subgoal 1''
Subgoal 1'''

Q.E.D.

The storage of TERMINATION-BOUND-SMT depends upon the :type-prescription
rule BOOLEANP-OF-MUST-RESPOND-P.

Summary
Form:  ( DEFTHM TERMINATION-BOUND-SMT ...)
Rules: ((:DEFINITION MUS

In [22]:
;; Theorem 4: Continue/respond are mutually exclusive and exhaustive
;; (when satisfaction is properly bounded)
(defthm continue-respond-partition-smt
  (implies (and (agent-state-p st)
                (>= (agent-state->satisfaction st) 0)
                (<= (agent-state->satisfaction st) 100))
           (or (must-respond-p st)
               (should-continue-p st)
               (>= (agent-state->satisfaction st) *satisfaction-threshold*)))
  :hints (("Goal"
           :smtlink
           (:fty (agent-state)))))


ACL2 Warning [Non-rec] in ( DEFTHM CONTINUE-RESPOND-PARTITION-SMT ...):
A :REWRITE rule generated from CONTINUE-RESPOND-PARTITION-SMT will
be triggered only by terms containing the function symbol NOT, which
has a non-recursive definition.  Unless this definition is disabled,
this rule is unlikely ever to be used.



Using clause-processor Smtlink
Goal'
Goal''
Goal'''
Goal'4'
Subgoal 2
Subgoal 2'
Subgoal 2.2
Subgoal 2.2'
Subgoal 2.2''
Subgoal 2.2'''
Using default SMT-trusted-cp...
; SMT solver: `python /tmp/py_file/smtlink.G72S6`: 0.06 sec, 0 bytes
Proved!
Subgoal 2.2'4'
Subgoal 2.2'5'
Subgoal 2.1
Subgoal 2.1'
Subgoal 1
Subgoal 1'
Subgoal 1''
Subgoal 1'''

Q.E.D.

The storage of CONTINUE-RESPOND-PARTITION-SMT depends upon the :type-
prescription rules BOOLEANP-OF-MUST-RESPOND-P and 
BOOLEANP-OF-SHOULD-CONTINUE-P.

Summary
Form:  ( DEFTHM CONTINUE-RESPOND-PARTITION-SMT ...)
Rules: ((:DEFINITION MUST-RESPOND-P)
        (:DEFINITION NOT)
        (:DEFINITION SHOULD-CONTINUE-P)
        (:

In [23]:
(defthm iteration-increases
  (implies (agent-state-p st)
           (< (agent-state->iteration st)
              (agent-state->iteration (increment-iteration st))))
  :hints (("Goal" :in-theory (enable increment-iteration))))

Goal'
Goal''
Goal'''

Q.E.D.

Summary
Form:  ( DEFTHM ITERATION-INCREASES ...)
Rules: ((:DEFINITION IFIX)
        (:DEFINITION INCREMENT-ITERATION)
        (:FAKE-RUNE-FOR-LINEAR NIL)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:REWRITE AGENT-STATE->ITERATION-OF-AGENT-STATE)
        (:TYPE-PRESCRIPTION AGENT-STATE->ITERATION$INLINE))
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.01)
Prover steps counted:  131
 ITERATION-INCREASES


In [24]:
(defthm deduct-preserves-agent-state
  (implies (and (tool-spec-p tool)
                (agent-state-p st))
           (agent-state-p (deduct-tool-cost tool st)))
  :hints (("Goal" :in-theory (enable deduct-tool-cost))))


ACL2 Warning [Subsume] in ( DEFTHM DEDUCT-PRESERVES-AGENT-STATE ...):
The previously added rule AGENT-STATE-P-OF-DEDUCT-TOOL-COST subsumes
a newly proposed :REWRITE rule generated from DEDUCT-PRESERVES-AGENT-STATE,
in the sense that the old rule rewrites a more general target.  Because
the new rule will be tried first, it may nonetheless find application.


Q.E.D.

The storage of DEDUCT-PRESERVES-AGENT-STATE depends upon the :compound-
recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM DEDUCT-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:DEFINITION DEDUCT-TOOL-COST)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.02)
Prover steps counted:  114
 DEDUCT-PRESERVES-AGENT-STATE


In [25]:
(defthm update-satisfaction-preserves-agent-state
  (implies (and (integerp new-sat)
                (agent-state-p st))
           (agent-state-p (update-satisfaction new-sat st)))
  :hints (("Goal" :in-theory (enable update-satisfaction))))


ACL2 Warning [Subsume] in ( DEFTHM UPDATE-SATISFACTION-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-UPDATE-SATISFACTION
subsumes a newly proposed :REWRITE rule generated from 
UPDATE-SATISFACTION-PRESERVES-AGENT-STATE, in the sense that the old
rule rewrites a more general target.  Because the new rule will be
tried first, it may nonetheless find application.


Q.E.D.

The storage of UPDATE-SATISFACTION-PRESERVES-AGENT-STATE depends upon
the :compound-recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-
prescription rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM UPDATE-SATISFACTION-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:DEFINITION UPDATE-SATISFACTION)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.02)
Prover steps counted:  47
 UPDATE-SATISFACTION-PRESERVES-AGENT-STATE


In [26]:
(defthm mark-done-preserves-agent-state
  (implies (agent-state-p st)
           (agent-state-p (mark-done st)))
  :hints (("Goal" :in-theory (enable mark-done))))


ACL2 Warning [Subsume] in ( DEFTHM MARK-DONE-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-MARK-DONE subsumes
a newly proposed :REWRITE rule generated from 
MARK-DONE-PRESERVES-AGENT-STATE, in the sense that the old rule rewrites
a more general target.  Because the new rule will be tried first, it
may nonetheless find application.


Q.E.D.

The storage of MARK-DONE-PRESERVES-AGENT-STATE depends upon the :compound-
recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM MARK-DONE-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:DEFINITION MARK-DONE)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.02)
Prover steps counted:  34
 MARK-DONE-PRESERVES-AGENT-STATE


In [27]:
(defthm increment-preserves-agent-state
  (implies (agent-state-p st)
           (agent-state-p (increment-iteration st)))
  :hints (("Goal" :in-theory (enable increment-iteration))))


ACL2 Warning [Subsume] in ( DEFTHM INCREMENT-PRESERVES-AGENT-STATE
...):  The previously added rule AGENT-STATE-P-OF-INCREMENT-ITERATION
subsumes a newly proposed :REWRITE rule generated from 
INCREMENT-PRESERVES-AGENT-STATE, in the sense that the old rule rewrites
a more general target.  Because the new rule will be tried first, it
may nonetheless find application.


Q.E.D.

The storage of INCREMENT-PRESERVES-AGENT-STATE depends upon the :compound-
recognizer rule CONSP-WHEN-AGENT-STATE-P and the :type-prescription
rule AGENT-STATE-P.

Summary
Form:  ( DEFTHM INCREMENT-PRESERVES-AGENT-STATE ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-AGENT-STATE-P)
        (:DEFINITION INCREMENT-ITERATION)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM)
        (:TYPE-PRESCRIPTION AGENT-STATE-P))
Warnings:  Subsume
Time:  0.02 seconds (prove: 0.01, print: 0.00, other: 0.01)
Prover steps counted:  37
 INCREMENT-PRESERVES-AGENT-STATE
